In [2]:
import pandas as pd
df = pd.read_excel("final data 11-22.xlsx")
df.head()

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,...,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,...,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,...,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,...,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,...,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43
4,32633,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,1231.166683,...,0.519048,0,1,5336,4,383,391,2114,1367.0,2022-08-10 16:49:00


In [10]:
df_sorted = df.sort_values(by=['Player_nickname', 'created_at']).groupby('Player_nickname')
df_sorted.head(3)

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,...,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,...,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,...,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,...,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,...,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43
20,47364,1,regular,STANDARD,NWL20,1200,0,CASUAL,1,372.547459,...,0.642857,1,0,11834,1,403,385,1901,1632.0,2022-08-25 03:33:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72753,31179,zuzu,regular,STANDARD,CSW21,1200,0,RATED,1,817.400729,...,0.561905,0,1,12354,2,328,410,1975,1850.0,2022-08-27 06:39:13
72754,10944,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,640.984386,...,0.551020,1,0,12358,3,504,340,1952,1870.0,2022-08-27 06:53:05
72759,66022,zyzzyva,regular,STANDARD,NWL20,1380,0,RATED,1,626.274697,...,0.595238,1,0,573,1,393,357,1859,1641.0,2022-07-28 18:47:59
72760,2581,STEEBot,regular,STANDARD,NWL20,1380,0,RATED,1,775.765387,...,0.571429,0,1,579,2,358,438,1863,1637.0,2022-07-28 18:58:36


In [146]:
# Creating historical average rating
import queue
import statistics


def historical_average_rating(df_sorted):
    base_rating = 1700
    
    score_df = pd.DataFrame(columns=['hist_player_score', 'game_id'])
    #Iterating through the games
    counter = 0
    group_count = 0
    games = 10
    for name, group_df in df_sorted:
        counter += 1

        # Initializing variables
        win = 0
        loss = 0
        games_played = 0
        
        # Creating a new queue
        oar_queue = queue.Queue()
        wins_queue = queue.Queue()
        for i in range(10):
            oar_queue.put(1700)
            wins_queue.put(0)
        
        for index, row in group_df.iterrows():
            
            # Getting details for 400(w-l)/g
            games_played = row['Player_game'] - 1
            
            
            
            # Adding fake games when less than 10, to have data
            if games_played == 0:
                fake_games = 10
                win_fake = 5
                loss_fake = 5
                win = 0
                loss = 0
            elif games_played < 10:
                fake_games = games-games_played
                win_fake = fake_games/2
                loss_fake = fake_games/2
                prev_10_win = get_prev_10_wins(wins_queue)
                prev_10_loss = games_played-prev_10_win
                win = prev_10_win + win_fake
                loss = prev_10_loss + loss_fake
            else:
                prev_10_win = get_prev_10_wins(wins_queue)
                prev_10_loss = games-prev_10_win
                win = prev_10_win
                loss = prev_10_loss
            
            #print(index)
            oar_score = get_oar_score(oar_queue)
            # Doing the calculation
            hist_score = oar_score + ((400)*(win-loss))/games

            score_df = pd.concat([score_df, pd.DataFrame({'hist_player_score': [hist_score], 'game_id': [row['game_id']]})], ignore_index=True)
            # After current game, update the variable with current game details
                # Updating bot rating, win/loss 
            oar_queue.put(row['Bot_rating'])
            removed_element = oar_queue.get()
            wins_queue.put(row['Player_win'])
            removed_element = wins_queue.get()
            
            
    return score_df
def get_oar_score(queue):
    
    values_list = list(queue.queue)
    oar = statistics.mean(values_list)
    return oar
    #opp_avg_rating = get_opp_avg_rating()

def get_prev_10_wins(wins_queue):
    result_sum = 0
    result_sum = sum(item for item in wins_queue.queue)
    
    return result_sum


In [147]:
score = historical_average_rating(df_sorted)
score

,hist_player_score,game_id
0,1700.0,43828
1,1700.0,5765
2,1691.2,40264
3,1677.8,44376
4,1676.8,32633
...,...,...
72758,1750.7,7264
72759,1700.0,66022
72760,1675.9,2581
72761,1732.2,54204


In [148]:
final_df = pd.merge(df, score, on='game_id', how='left')
final_df

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,...,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at,hist_player_score
0,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,...,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11,1700.0
1,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,...,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28,1700.0
2,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,...,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54,1691.2
3,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,...,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43,1677.8
4,32633,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,1231.166683,...,0,1,5336,4,383,391,2114,1367.0,2022-08-10 16:49:00,1676.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72768,15977,MasterBot,regular,STANDARD,CSW21,900,0,RATED,1,627.765249,...,2,0,23116,0,840,0,3975,0.0,2022-09-23 03:54:07,NaN
72769,53111,HastyBot,regular,STANDARD,CSW21,900,0,RATED,1,173.816351,...,2,0,23130,0,710,0,3964,0.0,2022-09-23 04:04:38,NaN
72770,35316,MasterBot,regular,STANDARD,CSW21,900,0,RATED,1,195.968033,...,2,0,23133,0,654,0,3959,0.0,2022-09-23 04:07:34,NaN
72771,20643,HastyBot,regular,STANDARD,CSW21,900,0,RATED,1,247.843816,...,2,0,23137,0,739,0,3952,0.0,2022-09-23 04:10:52,NaN


In [149]:
final_df.to_excel('hist_player_score_final_data_11-22.xlsx', index=False)